### Accessing multiple images using iMap API

#### Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests
%matplotlib inline
from PIL import Image
from io import BytesIO
import math
import json
from IPython.display import display, HTML

In [3]:
os.getcwd() # Get current working directory

'C:\\Users\\tmollick\\Documents\\iMapInvasives_NYNHP\\Code'

In [7]:
os.chdir(r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Datasets") # Change current working directory
# os.listdir() # Check new files in the new directory

In [9]:
# Reading the numbers from the text file
with open("presence_ids_with_photos.txt", "r") as file:
    presence_ids_with_photos = [int(line.strip()) for line in file]

# Print the list to verify
print(presence_ids_with_photos)


[1269970, 1391980, 1270879, 1271303, 1270987, 1270055, 1269928, 1323078, 1271037, 1271014, 1270929, 1351159, 1269959, 1269940, 1410198, 1271255, 1271035, 1249866, 1322554, 1271432, 1269887, 1271455, 1386214, 1322052, 1271224, 1323089, 1158522, 1270880, 1270945, 1270758, 1280894, 1045434, 1042327, 1435275, 1393477, 1413844, 1042333, 1417257, 1408840, 1341632, 1042362, 1271429, 1044113, 1331696, 1042546, 1409368, 1329118, 1443082, 1042329, 1042550, 1041713, 1441700, 1344384, 1041714, 1443662, 1230822, 1041716, 1331716, 1411656, 1441222, 1375427, 1269831, 1406991, 1157741, 1163998, 1443075, 1417261, 1444125, 1157735, 1389202, 1324094, 1410146, 1351791, 425375, 1163662, 1269855, 1388756, 1443092, 1390119, 1434743, 1444128, 1443080, 1390094, 510330, 1390065, 1390083, 1271094, 1443571, 1438376, 1271832, 1157793, 1390067, 1272705, 1146170, 1275086, 1037215, 1442164, 450686, 1042201, 1338922, 1279659, 1438606, 1281910, 1441690, 1335107, 1438378, 1417259, 1411655, 1298970, 1036859, 1438549, 450

In [5]:
presence_ids = presence_ids_with_photos

### Accessing iMapInvasives API https://www.imapinvasives.org/
#### iMap Username: tmollick@esf.edu
#### iMap Password: SUNY.esf@130436

### Accessing iNaturalist VisionAPI from https://rapidapi.com/inaturalist-inaturalist-default/api/visionapi
#### URL: https://rapidapi.com/hub
#### Username: nyimapinvasives
#### Email address: imapinvasives@dec.ny.gov
#### Password: Phragmity1!
#### Currently we have $20 subscription for 2000 image recognition in every month of VisionAPI

In [8]:
import requests

imap_site = "imapinvasives"
iMap_username = "tmollick@esf.edu"
iMap_password = "SUNY.esf@130436"

login_url = r"https://{0}.natureserve.org/imap/j_spring_security_check".format(imap_site)

# Attempt to log in
print("\nAttempting to authenticate with: {0}".format(login_url))
iMapSession = requests.Session()  # This is a global variable accessed in later functions
login_response = iMapSession.post(login_url,{'j_username':iMap_username,'j_password':iMap_password})
login_response_message = "\nlogin response: {0}".format(login_response.status_code)
print(login_response_message)
login_response.raise_for_status()

# Attempt to access a record to check if log-in was successful
test_aoi_url = r"https://{0}.natureserve.org/imap/services/aoi/new".format(imap_site)
test_aoi_record = iMapSession.get(test_aoi_url)
test_aoi_record_message = "\nTest record access response: {0}".format(test_aoi_record.status_code)
print(test_aoi_record_message)

if test_aoi_record.status_code == 403:
    print("\nResponse Code 403 is most likely the result of an incorrectly entered iMap username or password.  It may also be caused by logging in as a user with insufficient permissions.")

test_aoi_record.raise_for_status()


Attempting to authenticate with: https://imapinvasives.natureserve.org/imap/j_spring_security_check

login response: 200

Test record access response: 200


### Identification using VisionAPI (Exemption code)

#### This code saves progress incrementally by writing to the Excel file after processing each presence ID. If the script encounters an error or disconnection, it can be resumed without reprocessing the completed records.

In [10]:
import requests
import json
from PIL import Image
from io import BytesIO
import pandas as pd
import os
import time

# Function to extract latitude and longitude from the imap_dictionary
def get_lat_lon_from_imap(imap_dictionary):
    presence_point = imap_dictionary.get("presencePoint")
    if presence_point:
        return presence_point.get('latitude'), presence_point.get('longitude')

    presence_line = imap_dictionary.get("presenceLine")
    if presence_line:
        return presence_line.get('latitude'), presence_line.get('longitude')

    presence_polygon = imap_dictionary.get("presencePolygon")
    if presence_polygon:
        return presence_polygon.get('latitude'), presence_polygon.get('longitude')

    return None, None

# Function to reduce the image size in memory
def reduce_image_size(image, max_size_kb=700):
    exif_data = image.info.get('exif')
    quality = 95
    while True:
        buffer = BytesIO()
        if exif_data:
            image.save(buffer, "JPEG", quality=quality, exif=exif_data)
        else:
            image.save(buffer, "JPEG", quality=quality)
        if buffer.tell() <= max_size_kb * 1024 or quality <= 10:
            return buffer
        quality -= 5

# Function to identify species using iNaturalist VisionAPI
def identify_species(image_bytes, lat=None, lon=None):
    url = "https://visionapi.p.rapidapi.com/v1/rapidapi/score_image"
    headers = {
        "X-RapidAPI-Key": "fb2d499660mshffa90b4b45c3106p16583ejsnbaa19ef4277c",
        "X-RapidAPI-Host": "visionapi.p.rapidapi.com"
    }
    
    files = {'image': image_bytes}
    data = {}
    if lat is not None and lon is not None:
        data['lat'] = lat
        data['lng'] = lon
    
    response = requests.post(url, headers=headers, files=files, data=data)
    
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Function to get the image from a URL, handling redirects and setting headers
def get_image_from_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    response = requests.get(url, headers=headers, allow_redirects=True)
    
    if response.status_code == 200:
        try:
            return Image.open(BytesIO(response.content))
        except Exception as e:
            print(f"Failed to open image. Error: {e}")
            return None
    else:
        print(f"Failed to retrieve image. Status code: {response.status_code}")
        return None

# Function to process and identify species for all presentSpeciesId
def process_species_images(imap_dictionary, base_url):
    data_rows = []
    species_list = imap_dictionary.get("speciesList", [])
    
    for species in species_list:
        photo_url = None
        present_species_id = species.get("presentSpeciesId")
        presence_id = species.get("presenceId")
        imap_sci = species.get("nationalSpeciesList", {}).get("scientificName", "Unknown")
        imap_com = species.get("nationalSpeciesList", {}).get("commonName", "Unknown")
        
        # Extract imap_record_taxon
        imap_record_taxon = species.get("nationalSpeciesList", {}).get("inaturalistTaxonId", "Unknown")

        # Select only the first photo URL
        if "photos" in species and species["photos"]:
            photo_url = species["photos"][0].get("photoUrl", "")
        
        if photo_url:
            img = get_image_from_url(photo_url)
            if img:
                resized_image_buffer = reduce_image_size(img, max_size_kb=700)
                lat, lon = get_lat_lon_from_imap(imap_dictionary)
                resized_image_buffer.seek(0)
                result = identify_species(resized_image_buffer, lat, lon)
                
                if result and "results" in result and result["results"]:
                    # Select the best result based on combined score
                    top_result = max(result["results"], key=lambda x: x["original_combined_score"])
                    inat_sci = top_result['taxon']['name']
                    inat_com = top_result['taxon'].get('preferred_common_name', 'Unknown').capitalize()
                    inat_taxon = top_result['taxon']['id']
                    geo_score = round(top_result['original_geo_score'], 2)
                    com_score = round(top_result['original_combined_score'], 2)
                    inatlink_html = f'=HYPERLINK("https://www.inaturalist.org/taxa/{inat_taxon}-{inat_sci.replace(" ", "-")}", "View")'
                else:
                    inat_sci = "Unknown"
                    inat_com = "Unknown"
                    inat_taxon = "Unknown"
                    geo_score = "Unknown"
                    com_score = "Unknown"
                    inatlink_html = "Unknown"
            else:
                photo_url = "No Photo"
                inat_sci = "Unknown"
                inat_com = "Unknown"
                inat_taxon = "Unknown"
                geo_score = "Unknown"
                com_score = "Unknown"
                inatlink_html = "Unknown"
        else:
            photo_url = "No Photo"
            inat_sci = "Unknown"
            inat_com = "Unknown"
            inat_taxon = "Unknown"
            geo_score = "Unknown"
            com_score = "Unknown"
            inatlink_html = "Unknown"
            
        species_label = 1 if str(imap_record_taxon) == str(inat_taxon) else 0
        imaplink = f"https://imapinvasives.natureserve.org/imap/services/page/Presence/{presence_id}.html"
        imaplink_html = f'=HYPERLINK("{imaplink}", "View")'
        iMapPhoto_html = f'=HYPERLINK("{photo_url}", "View")' if photo_url != "No Photo" else "No Photo"
        
        row = [None, imaplink_html, presence_id, present_species_id, iMapPhoto_html, imap_sci, imap_com, imap_record_taxon, inatlink_html, inat_sci, inat_com, inat_taxon, geo_score, com_score, "", species_label, ""]
        data_rows.append(row)
    
    return data_rows

# Process all the presence IDs
def process_multiple_species_ids(presence_ids):
    base_url = "https://imapinvasives.natureserve.org/imap/services/presence/"  # Keeping the base URL unchanged for the API
    output_file = r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Output_unconfirmed_30_records_per_9_species.xlsx"
    
    # Load existing data if the file exists
    if os.path.exists(output_file):
        df_existing = pd.read_excel(output_file)
        processed_ids = set(df_existing["presenceId"].dropna().astype(int))
    else:
        df_existing = pd.DataFrame(columns=[
            "S.L.", "imaplink", "presenceId", "presentSpeciesId", "iMapPhoto", "imap_sci",
            "imap_com", "imap_record_taxon", "inatlink", "inat_sci", "inat_com", "inat_taxon", 
            "geo_score", "com_score", "visual_model", "species_label", "com_status"
        ])
        processed_ids = set()

    for idx, presence_id in enumerate(presence_ids, start=1):
        if presence_id in processed_ids:
            print(f"Skipping already processed presence ID: {presence_id}")
            continue

        print(f"\nProcessing presence ID: {presence_id}")
        full_url = f"{base_url}{presence_id}"
        
        while True:
            try:
                imap_record = iMapSession.get(full_url)  # Using iMapSession to access the API
                
                if imap_record.status_code == 200:
                    imap_dictionary = imap_record.json()
                    data_rows = process_species_images(imap_dictionary, base_url)
                    for row in data_rows:
                        row[0] = idx  # Insert S.L. as the first column
                        df_existing.loc[len(df_existing)] = row
                    # Save progress after each record
                    df_existing.to_excel(output_file, index=False)
                    processed_ids.add(presence_id)
                else:
                    print(f"Failed to retrieve data for presence ID {presence_id}. Status code: {imap_record.status_code}")
                break
            except requests.exceptions.RequestException as e:
                print(f"Error processing presence ID {presence_id}: {e}. Retrying in 5 seconds...")
                time.sleep(5)
            except Exception as e:
                print(f"Unexpected error: {e}. Saving progress and exiting.")
                df_existing.to_excel(output_file, index=False)
                return

# Example usage: Replace presence_ids with your actual list of IDs
process_multiple_species_ids(presence_ids)


Processing presence ID: 1269970

Processing presence ID: 1391980

Processing presence ID: 1270879

Processing presence ID: 1271303

Processing presence ID: 1270987

Processing presence ID: 1270055

Processing presence ID: 1269928

Processing presence ID: 1323078

Processing presence ID: 1271037

Processing presence ID: 1271014

Processing presence ID: 1270929

Processing presence ID: 1351159

Processing presence ID: 1269959

Processing presence ID: 1269940

Processing presence ID: 1410198

Processing presence ID: 1271255

Processing presence ID: 1271035

Processing presence ID: 1249866

Processing presence ID: 1322554

Processing presence ID: 1271432

Processing presence ID: 1269887

Processing presence ID: 1271455

Processing presence ID: 1386214

Processing presence ID: 1322052

Processing presence ID: 1271224

Processing presence ID: 1323089

Processing presence ID: 1158522

Processing presence ID: 1270880

Processing presence ID: 1270945

Processing presence ID: 1270758

Processin

### Apply threshold values to the output file from iNaturalist

In [9]:
import pandas as pd
from openpyxl import load_workbook, Workbook

df = pd.read_excel(r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Reviewed_Output_unconfirmed_30_records_per_9_species.xlsx", engine="openpyxl")
df.head()

,S.L.,imaplink,presenceId,presentSpeciesId,iMapPhoto,imap_sci,imap_com,imap_record_taxon,inatlink,inat_sci,inat_com,inat_taxon,geo_score,com_score,visual_model,species_label,com_status
0,1,View,1269970,1278901,View,Adelges tsugae,Hemlock Woolly Adelgid,61513.0,View,Adelges tsugae,Hemlock woolly adelgid,61513,32.92,78.56,NaN,1,NaN
1,2,View,1391980,1410113,View,Adelges tsugae,Hemlock Woolly Adelgid,61513.0,View,Adelges tsugae,Hemlock woolly adelgid,61513,15.53,64.56,NaN,1,NaN
2,3,View,1270879,1279819,View,Adelges tsugae,Hemlock Woolly Adelgid,61513.0,View,Tsuga canadensis,Eastern hemlock,48734,63.3,68.28,NaN,0,NaN
3,4,View,1271303,1280268,View,Adelges tsugae,Hemlock Woolly Adelgid,61513.0,View,Tsuga canadensis,Eastern hemlock,48734,63.3,48.3,NaN,0,NaN
4,5,View,1270987,1279927,View,Adelges tsugae,Hemlock Woolly Adelgid,61513.0,View,Tsuga canadensis,Eastern hemlock,48734,63.3,39.82,NaN,0,NaN


### Added thredhold and Geo Score

In [12]:
import pandas as pd
df2  = pd.read_excel(r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Raw_datasets\Tiers of NYS invasive Species.xlsx", engine="openpyxl")
df2.head()

,Common Name,Scientific Name,Type,Ecological,Socio-Economic,NYS Part 575,NYS_Tier,APIPP,Capital Region,CRISP,Finger Lakes,Lower Hudson,LISMA,SLELO,WNY
0,African clawed frog,Xenopus laevis,AA,Moderate,Insignificant Positive,Regulated,NaN,NaN,NaN,NaN,NaN,M,NaN,NaN,NaN
1,African elodea,Lagarosiphon major,AP,High,Low Negative,NaN,1b,NaN,NaN,NaN,NaN,1c,NaN,NaN,NaN
2,Africanized honey bee,"Apis mellifera scutellata x A. m. ligustica, A...",TA,Not assessed,Not assessed,Prohibited,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN
3,Agrilus sp. 9895,Agrilus sp. 9895,TA,Not assessed,Not assessed,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alewife,Alosa pseudoharengus,AA,Moderate,Insignificant Negative,NaN,NaN,NaN,2,NaN,4,NaN,NaN,NaN,4


In [2]:
import pandas as pd
from openpyxl import load_workbook, Workbook

# Define file paths
input_file = r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Reviewed_Output_unconfirmed_30_records_per_9_species.xlsx"
output_file = r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Final_species_identification_9_species.xlsx"

# Load the original Excel file using OpenPyXL
wb = load_workbook(input_file, data_only=False)
ws = wb.active

# Load the dataset using Pandas (this will lose hyperlinks but is needed for calculations)
df = pd.read_excel(input_file, engine="openpyxl")

# Convert columns to numeric
df["com_score"] = pd.to_numeric(df["com_score"], errors="coerce")
df["geo_score"] = pd.to_numeric(df["geo_score"], errors="coerce")  # Convert geo_score to numeric

# Define threshold values for the 11 specific species
threshold_values = {
    "Litylenchus crenatae mccannii": 86.69,
    "Rhamnus cathartica": 81.5,
    "Alliaria petiolata": 88.91,
    "Adelges tsugae": 82.85,
    "Japanese Knotweed": 76.73,
    "Berberis thunbergii": 90.06,
    "Rosa multiflora": 74.05,
    "Celastrus orbiculatus": 65.45,
    "Lythrum salicaria": 89.08,
    "Lycorma delicatula": 92.79,
    "Ailanthus altissima": 87.09
}
default_threshold = 77.62  # Default threshold for other species

# Define geo score threshold values
Geo_score_mean = {
    "Litylenchus crenatae mccannii": 21.54,
    "Rhamnus cathartica": 31.47,
    "Alliaria petiolata": 39.89,
    "Adelges tsugae": 21.26,
    "Reynoutria japonica": 31.94,  # Updated from "Japanese Knotweed"
    "Berberis thunbergii": 36.55,
    "Rosa multiflora": 41.31,
    "Celastrus orbiculatus": 33.65,
    "Lythrum salicaria": 38.21,
    "Lycorma delicatula": 72.02,
    "Ailanthus altissima": 32.68
}
default_geo_score = 36.5  # Default geo score threshold for other species

# Assign species-specific thresholds
df["Species_selected_threshold"] = df["imap_sci"].apply(lambda x: threshold_values.get(x, default_threshold))
df["Geo_score_mean"] = df["imap_sci"].apply(lambda x: Geo_score_mean.get(x, default_geo_score))

# Compare 'com_score' with the threshold and assign "Above" or "Below"
df["Com_score_above_or_below"] = df.apply(
    lambda row: "Above" if row["com_score"] >= row["Species_selected_threshold"] else "Below", axis=1
)

# Compare 'geo_score' with the threshold and assign "Above" or "Below"
df["Geo_score_above_or_below"] = df.apply(
    lambda row: "Above" if row["geo_score"] >= row["Geo_score_mean"] else "Below", axis=1
)

# Apply exception rule based on 'species_label'
df["com_status"] = df["species_label"].apply(lambda x: "Match" if x == 1 else "Unmatch")

# Assign "Recommended_status" based on the given conditions
df["Recommended_status"] = "Manual review"  # Default value

# Assign "Automatically confirmed" only when all conditions are met
df.loc[
    (df["com_score"] >= df["Species_selected_threshold"]) & 
    (df["com_status"] == "Match") & 
    (df["geo_score"] >= df["Geo_score_mean"]),
    "Recommended_status"
] = "Automatically confirmed"

# Preserve hyperlinks using `values_only=False`
hyperlink_columns = ["imaplink", "iMapPhoto", "inatlink"]

# Create a new workbook for writing
wb_new = Workbook()
ws_new = wb_new.active
ws_new.title = "Processed Data"

# Copy headers (Include both original and new columns)
headers = list(df.columns)
ws_new.append(headers)

# Loop through rows and copy data while preserving hyperlinks
for row_idx, row in enumerate(ws.iter_rows(values_only=False), start=1):
    if row_idx == 1:  # Skip header row (already written)
        continue

    new_row = []  # Store the new row values
    row_data = {}  # Store column-wise data

    for col_idx, cell in enumerate(row, start=1):
        column_name = ws.cell(row=1, column=col_idx).value  # Get column name from header row

        if cell.hyperlink:  # Preserve hyperlinks exactly as in the input file
            new_cell = ws_new.cell(row=row_idx, column=col_idx)
            new_cell.value = cell.value
            new_cell.hyperlink = cell.hyperlink.target
            new_cell.style = "Hyperlink"
        else:
            row_data[column_name] = cell.value

    # Add the data from Pandas DataFrame (Ensuring correct column alignment)
    for col_name in ["Species_selected_threshold", "Com_score_above_or_below", "Geo_score_mean", "Geo_score_above_or_below", "Recommended_status", "com_status"]:
        row_data[col_name] = df.at[row_idx - 2, col_name]

    # Ensure the new row is in the correct order
    new_row = [row_data.get(col, None) for col in headers]

    # Append the correctly formatted row
    ws_new.append(new_row)

# Save the final workbook with hyperlinks intact
wb_new.save(output_file)

print(f"Processing complete. File saved at: {output_file} with hyperlinks preserved and all data correctly aligned.")


Processing complete. File saved at: C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Final_species_identification_9_species.xlsx with hyperlinks preserved and all data correctly aligned.


---

### New York State Tier of species link: https://www.nynhp.org/invasives/species-tiers-table/

In [29]:
import pandas as pd
from openpyxl import load_workbook, Workbook

# Define file paths
input_file = r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Dataset_11_species\11 Species.xlsx"
df2_file = r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Raw_datasets\Tiers of NYS invasive Species.xlsx"
output_file = r"C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Final_species_identification_9_species_V3.xlsx"

# Load the original Excel file using OpenPyXL and Pandas
wb = load_workbook(input_file, data_only=False)
ws = wb.active
df = pd.read_excel(input_file, engine="openpyxl")

# Load df2 with tier info
df2 = pd.read_excel(df2_file, engine="openpyxl")

# Convert columns to numeric where necessary
df["com_score"] = pd.to_numeric(df["com_score"], errors="coerce")
df["geo_score"] = pd.to_numeric(df["geo_score"], errors="coerce")

# Threshold values for com_score and geo_score
threshold_values = {
    "Litylenchus crenatae mccannii": 86.69,
    "Rhamnus cathartica": 81.5,
    "Alliaria petiolata": 88.91,
    "Adelges tsugae": 82.85,
    "Japanese Knotweed": 76.73,
    "Berberis thunbergii": 90.06,
    "Rosa multiflora": 74.05,
    "Celastrus orbiculatus": 65.45,
    "Lythrum salicaria": 89.08,
    "Lycorma delicatula": 92.79,
    "Ailanthus altissima": 87.09
}
default_threshold = 77.62

Geo_score_mean = {
    "Litylenchus crenatae mccannii": 21.54,
    "Rhamnus cathartica": 31.47,
    "Alliaria petiolata": 39.89,
    "Adelges tsugae": 21.26,
    "Reynoutria japonica": 31.94,
    "Berberis thunbergii": 36.55,
    "Rosa multiflora": 41.31,
    "Celastrus orbiculatus": 33.65,
    "Lythrum salicaria": 38.21,
    "Lycorma delicatula": 72.02,
    "Ailanthus altissima": 32.68
}
default_geo_score = 36.5

# Merge df with df2 to get the NYS_Tier info
df = df.merge(df2[['Scientific Name', 'NYS_Tier']], how='left', left_on='imap_sci', right_on='Scientific Name')

# Assign thresholds
df["Species_selected_threshold"] = df["imap_sci"].apply(lambda x: threshold_values.get(x, default_threshold))
df["Geo_score_mean"] = df["imap_sci"].apply(lambda x: Geo_score_mean.get(x, default_geo_score))

# Check com_score status (applies to everyone)
df["Com_score_above_or_below"] = df.apply(
    lambda row: "Above" if row["com_score"] >= row["Species_selected_threshold"] else "Below", axis=1
)

# Check geo_score status
df["Geo_score_above_or_below"] = df.apply(
    lambda row: "Above" if row["geo_score"] >= row["Geo_score_mean"] else "Below", axis=1
)

# Apply exception rule based on 'species_label'
df["com_status"] = df["species_label"].apply(lambda x: "Match" if x == 1 else "Unmatch")

# Default value for Recommended_status
df["Recommended_status"] = "Manual review"

# Clean up NYS_Tier values (no conversion to numeric because there are non-numeric tiers like 1a, 1b, etc.)
df["NYS_Tier"] = df["NYS_Tier"].astype(str).str.strip()

# Define the tiers where geo_score applies
tiers_with_geo_score = ['1a', '1b', '1c', '1', '2', '3']

# Condition for Tiers 1a, 1b, 1c, 1, 2, 3 -> apply com_score + geo_score + match
tier_1_to_3_condition = (
    (df["NYS_Tier"].isin(tiers_with_geo_score)) &
    (df["com_score"] >= df["Species_selected_threshold"]) &
    (df["geo_score"] >= df["Geo_score_mean"]) &
    (df["com_status"] == "Match")
)

# Condition for Tier 4 and blanks -> apply com_score + match only
tier_4_or_blank_condition = (
    (~df["NYS_Tier"].isin(tiers_with_geo_score)) &
    (df["com_score"] >= df["Species_selected_threshold"]) &
    (df["com_status"] == "Match")
)

# Apply conditions
df.loc[tier_1_to_3_condition | tier_4_or_blank_condition, "Recommended_status"] = "Automatically confirmed"

# Drop 'Scientific Name' and any other unnecessary columns before export
df.drop(columns=["Scientific Name", "visual_model"], inplace=True, errors='ignore')

# Preserve hyperlinks using `values_only=False`
hyperlink_columns = ["imaplink", "iMapPhoto", "inatlink"]

# Create a new workbook for writing
wb_new = Workbook()
ws_new = wb_new.active
ws_new.title = "Processed Data"

# Copy headers (Include both original and new columns)
headers = list(df.columns)
ws_new.append(headers)

# Loop through rows and copy data while preserving hyperlinks
for row_idx, row in enumerate(ws.iter_rows(values_only=False), start=1):
    if row_idx == 1:  # Skip header row (already written)
        continue

    new_row = []
    row_data = {}

    for col_idx, cell in enumerate(row, start=1):
        column_name = ws.cell(row=1, column=col_idx).value

        # Skip columns we dropped earlier
        if column_name in ["Scientific Name", "visual_model"]:
            continue

        if cell.hyperlink:
            new_cell = ws_new.cell(row=row_idx, column=col_idx)
            new_cell.value = cell.value
            new_cell.hyperlink = cell.hyperlink.target
            new_cell.style = "Hyperlink"
        else:
            row_data[column_name] = cell.value

    # Add the data from Pandas DataFrame
    for col_name in [
        "Species_selected_threshold", "Com_score_above_or_below", "Geo_score_mean",
        "Geo_score_above_or_below", "Recommended_status", "com_status", "NYS_Tier"
    ]:
        row_data[col_name] = df.at[row_idx - 2, col_name]

    # Ensure the new row is in the correct order (excluding dropped columns)
    new_row = [row_data.get(col, None) for col in headers]

    ws_new.append(new_row)

# Save the final workbook
wb_new.save(output_file)

print(f"Processing complete. File saved at: {output_file} with hyperlinks preserved and tier-specific geo_score applied.")


Processing complete. File saved at: C:\Users\tmollick\Documents\iMapInvasives_NYNHP\Outputs\Final_species_identification_9_species_V3.xlsx with hyperlinks preserved and tier-specific geo_score applied.
